In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /home/joepareti54/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pwd

/media/joepareti54/Elements/x/finance-2021/AI/NLP/adavanced


In [ ]:
#!pip install google-colab

In [ ]:
#from google.colab import files
#uploaded = files.upload()
#df = pd.read_csv("tennis.csv")
#df.head()

In [3]:
text = """The Day Coronavirus Nearly Broke the Financial Markets.
The March 16 stock crash was part of a broader liquidity crisis that threatened the viability of
America’s companies and municipalities.
An urgent call reached Ronald O’Hanley, State Street Corp.’s chief executive, as he
sat in his oﬃce in downtown Boston. It was 8 a.m. on Monday, March 16.
A senior deputy told him corporate treasurers and pension managers, panicked by
the growing economic damage from the Covid-19 pandemic, were pulling billions
of dollars from certain money-market funds. This was forcing the funds to try to
sell some of the bonds they held.
But there were almost no buyers. Everybody was suddenly desperate for cash.
He and the deputy, asset-management executive Cyrus Taraporevala, had spoken
the night before, wrestling with how investors would respond to an emergency
interest-rate cut from the Federal Reserve.
Now, they had their answer. In his 34 years in ﬁnance, Mr. O’Hanley had
weathered plenty of meltdowns, but never one like this.
“The market is fearing the worst,” Mr. O’Hanley told him.
March 16 was the day a microscopic virus brought the ﬁnancial system to the
brink. Few realized how close it came to going over the edge entirely.
The Dow Jones Industrial Average plunged nearly 13% that day, the second biggest one-day fall in history. 
Stock-market volatility spiked to a record high.
Investors struggled to unload even safe bonds, like Treasurys. Companies and
government oﬃcials were losing access to the lending markets on which they rely
to make payroll and build schools.
"""

In [4]:
from nltk.tokenize import sent_tokenize
sentences = []

sentences.append(sent_tokenize(text))

sentences = [y for x in sentences for y in x]

In [6]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

/tmp/ipykernel_9269/2999140426.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


In [7]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [8]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [9]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [11]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(5):
  print("ARTICLE:")
#  print(df['article_text'][i])
  print('\n')
  print("SUMMARY:")
  print(ranked_sentences[i][1])
  print('\n')

ARTICLE:


SUMMARY:
He and the deputy, asset-management executive Cyrus Taraporevala, had spoken
the night before, wrestling with how investors would respond to an emergency
interest-rate cut from the Federal Reserve.


ARTICLE:


SUMMARY:
A senior deputy told him corporate treasurers and pension managers, panicked by
the growing economic damage from the Covid-19 pandemic, were pulling billions
of dollars from certain money-market funds.


ARTICLE:


SUMMARY:
The March 16 stock crash was part of a broader liquidity crisis that threatened the viability of
America’s companies and municipalities.


ARTICLE:


SUMMARY:
The Day Coronavirus Nearly Broke the Financial Markets.


ARTICLE:


SUMMARY:
The Dow Jones Industrial Average plunged nearly 13% that day, the second biggest one-day fall in history.


